** This is a work in progress **
# Objective:
1.	Analysing the Titanic dataset to obtain useful and interesting insights.
2.	Predicting survival using multiple ML techniques
After poking around for inspiration from fellow kagglers, i noticed Random Forest Trees to be the go-to technique for prediction in this competition.. Having python at my disposal, i figured i am interested in attempting multiple ML techniques and hope to analyze how they fare.

In [ ]:
# Data Analysis and Wrangling
import numpy as np 
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# allowing multiple scrollable outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
# Load data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

# To perform manipulation that both datasets need in common
full = train.append(test,ignore_index = True) 


Now that we have our data ready, we can look at the variables, shape of the data, missing values and some standard descriptive stats. Further, we can figure out sensible data cleaning and possible feature extraction/engineering opportunities.

In [ ]:
# preliminary analysis
full.isnull().sum()